In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, ConcatDataset
import copy
import json

In [3]:

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [4]:
def add_repetitive_data(train_dataset, percentage):
    """
    Adds repetitive data to the training set based on the given percentage.
    
    Args:
        train_dataset: Original training dataset.
        percentage: Percentage of repetitive data to add (e.g., 10, 20, ..., 100).
    
    Returns:
        A new dataset with repetitive data added.
    """
    repeat_count = percentage // 10
    repetitive_data = copy.deepcopy(train_dataset)
    
    # Create a dataset with repetitive data
    repetitive_datasets = [train_dataset]
    for _ in range(repeat_count):
        repetitive_datasets.append(repetitive_data)
    
    return ConcatDataset(repetitive_datasets)


In [ ]:
# standard training with different percentages of repetition (30 epock)

In [ ]:
# Step 1: Load CIFAR-10 Dataset
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Step 2: Define Data Loaders
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

# Step 3: Initialize SimpleCNN Model
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# Training with different levels of repetitive data
num_epochs = 30  # Reduce the number of epochs to speed up training
repetition_levels = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

results = []

for repetition in repetition_levels:
    print(f"\nTraining with {repetition}% repetitive data")
    
    # Create repetitive dataset
    repetitive_train_dataset = add_repetitive_data(train_dataset, repetition)
    train_loader = DataLoader(repetitive_train_dataset, batch_size=128, shuffle=True, num_workers=4)
    
    # Reset model weights before training
    model.apply(lambda m: m.reset_parameters() if hasattr(m, 'reset_parameters') else None)
    
    # Train the model
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            
            # Standard forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')
    
    # Evaluate on test set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    test_accuracy = correct / total
    print(f'Test Accuracy: {100 * test_accuracy:.2f}%')
    
    # Evaluate on repetitive training set
    train_loader_eval = DataLoader(repetitive_train_dataset, batch_size=128, shuffle=False)
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in train_loader_eval:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    repetitive_train_accuracy = correct / total
    print(f'Repetitive Train Set Accuracy: {100 * repetitive_train_accuracy:.2f}%')
    
    # Evaluate on non-repetitive (original) training set
    train_loader_eval = DataLoader(train_dataset, batch_size=128, shuffle=False)
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in train_loader_eval:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    non_repetitive_train_accuracy = correct / total
    print(f'Non-Repetitive Train Set Accuracy: {100 * non_repetitive_train_accuracy:.2f}%')
    
    # Save the results
    results.append({
        'repetition': repetition,
        'test_accuracy': test_accuracy,
        'repetitive_train_accuracy': repetitive_train_accuracy,
        'non_repetitive_train_accuracy': non_repetitive_train_accuracy
    })

# Save results to a JSON file
with open('training_results_no_attack.json', 'w') as f:
    json.dump(results, f, indent=4)

print("\nTraining with different levels of repetitive data completed! Results saved to 'training_results_no_attack.json'.")

Files already downloaded and verified
Files already downloaded and verified

Training with 0% repetitive data
Epoch [1/30], Loss: 1.9787
Epoch [2/30], Loss: 1.6727
Epoch [3/30], Loss: 1.5519
Epoch [4/30], Loss: 1.4648
Epoch [5/30], Loss: 1.3933
Epoch [6/30], Loss: 1.3248
Epoch [7/30], Loss: 1.2815
Epoch [8/30], Loss: 1.2389
Epoch [9/30], Loss: 1.2094
Epoch [10/30], Loss: 1.1812
Epoch [11/30], Loss: 1.1572
Epoch [12/30], Loss: 1.1345
Epoch [13/30], Loss: 1.1212
Epoch [14/30], Loss: 1.1011
Epoch [15/30], Loss: 1.0919
Epoch [16/30], Loss: 1.0708
Epoch [17/30], Loss: 1.0638
Epoch [18/30], Loss: 1.0529
Epoch [19/30], Loss: 1.0365
Epoch [20/30], Loss: 1.0242
Epoch [21/30], Loss: 1.0210
Epoch [22/30], Loss: 1.0130
Epoch [23/30], Loss: 1.0021
Epoch [24/30], Loss: 0.9966
Epoch [25/30], Loss: 0.9940
Epoch [26/30], Loss: 0.9816
Epoch [27/30], Loss: 0.9748
Epoch [28/30], Loss: 0.9759
Epoch [29/30], Loss: 0.9621
Epoch [30/30], Loss: 0.9630
Test Accuracy: 70.72%
Repetitive Train Set Accuracy: 71.75%

In [ ]:
#Adversarial setting (adversarial training with different percentages of repetition (10 epock))

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
from tqdm import tqdm

# Parameters
epsilon = 0.5  # PGD attack strength
repetition_ratios = [0.1 * i for i in range(0, 11)]  # 10%, 20%, ..., 100%
num_epochs = 30
batch_size = 128
learning_rate = 0.001

def pgd_attack(model, images, labels, epsilon, alpha=0.01, iters=10):
    images = images.clone().detach().to(images.device)
    original_images = images.clone().detach()

    for _ in range(iters):
        images.requires_grad = True
        outputs = model(images)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        model.zero_grad()
        loss.backward()
        adv_images = images + alpha * images.grad.sign()
        eta = torch.clamp(adv_images - original_images, min=-epsilon, max=epsilon)
        images = torch.clamp(original_images + eta, min=0, max=1).detach()

    return images

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = nn.ReLU()(self.conv1(x))
        x = nn.MaxPool2d(2, 2)(x)
        x = nn.ReLU()(self.conv2(x))
        x = nn.MaxPool2d(2, 2)(x)
        x = x.view(-1, 64 * 8 * 8)
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_data = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

# DataLoader for test set
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

def evaluate_model(model, loader, adversarial=False):
    model.eval()
    correct = 0
    total = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        if adversarial:
            images = pgd_attack(model, images, labels, epsilon)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    return correct / total

# Prepare device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Train and evaluate with varying repetition ratios
results = []
for ratio in repetition_ratios:
    print(f"Training with {int(ratio * 100)}% repetitive data...")

    # Add repetitive data
    n_repeat = int(len(train_data) * ratio)
    repetitive_indices = np.random.choice(len(train_data), n_repeat, replace=False)
    full_indices = list(range(len(train_data))) + list(repetitive_indices)
    augmented_data = Subset(train_data, full_indices)
    train_loader = DataLoader(augmented_data, batch_size=batch_size, shuffle=True)

    # DataLoader for non-repeated training set
    non_repeated_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False)

    # Initialize model, loss, and optimizer
    model = SimpleCNN().to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # Adversarial training
    for epoch in range(num_epochs):
        model.train()
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
            images, labels = images.to(device), labels.to(device)

            # Generate adversarial examples
            adv_images = pgd_attack(model, images, labels, epsilon)

            # Forward pass
            outputs = model(adv_images)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # Evaluate on test set
    test_acc = evaluate_model(model, test_loader)
    adv_test_acc = evaluate_model(model, test_loader, adversarial=True)

    # Evaluate on repeated training set
    train_acc = evaluate_model(model, train_loader)
    adv_train_acc = evaluate_model(model, train_loader, adversarial=True)

    # Evaluate on non-repeated training set
    non_repeated_train_acc = evaluate_model(model, non_repeated_loader)
    adv_non_repeated_train_acc = evaluate_model(model, non_repeated_loader, adversarial=True)

    results.append((
        ratio, 
        train_acc, adv_train_acc, 
        non_repeated_train_acc, adv_non_repeated_train_acc, 
        test_acc, adv_test_acc
    ))

    print(f"Repetition Ratio: {int(ratio * 100)}%, "
          f"Train Acc: {train_acc:.4f}, Adv Train Acc: {adv_train_acc:.4f}, "
          f"Non-Repeated Train Acc: {non_repeated_train_acc:.4f}, Adv Non-Repeated Train Acc: {adv_non_repeated_train_acc:.4f}, "
          f"Test Acc: {test_acc:.4f}, Adv Test Acc: {adv_test_acc:.4f}")

# Save results
import csv
with open("All_results.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow([
        "Repetition Ratio", "Train Accuracy", "Adversarial Train Accuracy", 
        "Non-Repeated Train Accuracy", "Adversarial Non-Repeated Train Accuracy", 
        "Test Accuracy", "Adversarial Test Accuracy"
    ])
    writer.writerows(All_results)

print("Results saved to All_results.csv.")
# Training with 0% repetitive data Repetition Ratio: 0%, Train Acc: 0.3793, Adv Train Acc: 0.2386, Non-Repeated Train Acc: 0.3793, Adv Non-Repeated Train Acc: 0.2386, Test Acc: 0.3739, Adv Test Acc: 0.2433
# Training with 100% repetitive dataRepetition Ratio: 100%, Train Acc: 0.4256, Adv Train Acc: 0.2674, Non-Repeated Train Acc: 0.4256, Adv Non-Repeated Train Acc: 0.2674, Test Acc: 0.4138, Adv Test Acc: 0.2517


Files already downloaded and verified
Files already downloaded and verified
Training with 0% repetitive data...


Epoch 30/30: 100%|██████████| 391/391 [00:12<00:00, 31.55it/s]


Repetition Ratio: 0%, Train Acc: 0.4135, Adv Train Acc: 0.2530, Non-Repeated Train Acc: 0.4135, Adv Non-Repeated Train Acc: 0.2530, Test Acc: 0.4116, Adv Test Acc: 0.2503
Training with 10% repetitive data...


Epoch 30/30: 100%|██████████| 430/430 [00:13<00:00, 31.48it/s]


Repetition Ratio: 10%, Train Acc: 0.3448, Adv Train Acc: 0.2577, Non-Repeated Train Acc: 0.3435, Adv Non-Repeated Train Acc: 0.2565, Test Acc: 0.3376, Adv Test Acc: 0.2469
Training with 20% repetitive data...


...

Epoch 30/30: 100%|██████████| 743/743 [00:23<00:00, 31.35it/s]


Repetition Ratio: 90%, Train Acc: 0.3947, Adv Train Acc: 0.2580, Non-Repeated Train Acc: 0.3947, Adv Non-Repeated Train Acc: 0.2577, Test Acc: 0.3937, Adv Test Acc: 0.2549
Training with 100% repetitive data...


Epoch 30/30: 100%|██████████| 782/782 [00:24<00:00, 31.42it/s]


Repetition Ratio: 100%, Train Acc: 0.3577, Adv Train Acc: 0.2539, Non-Repeated Train Acc: 0.3577, Adv Non-Repeated Train Acc: 0.2539, Test Acc: 0.3621, Adv Test Acc: 0.2538


NameError: name 'All_results' is not defined